In [14]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from urllib.request import urlopen
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
import time

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

In [ ]:
import mysql.connector
remote = mysql.connector.connect(
    host = "****",
    port = 3306,
    user = "root",
    password = "****",
    database = "****"
)

In [9]:
cur = remote.cursor()

In [11]:
class Info:
    def __init__(self):
        self.brand_name = None
        self.model_name = None
        self.price = None
        self.accident = None
        self.year = None
        self.distance = None
        self.cc = None
        self.kind = None
        self.transmission = None
        self.fueltype = None

    def sql_query(self):
        sql = "insert into Info(brand, name, price, accident, year, distance, cc, kind, transmission, fueltype) values (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
        return sql
    
    def __del__(self):
        print("=*= Car Deleted =*=")

In [ ]:
url = "http://www.encar.com/fc/fc_carsearchlist.do?carType=for#!"
driver.get(url)

driver.maximize_window()
pages_list = driver.find_element(By.ID, "pagination")
pages = pages_list.find_elements(By.TAG_NAME, "span")
pages = pages[:-1]

for i in range(len(pages)):
    pages_list = driver.find_element(By.ID, "pagination")
    pages = pages_list.find_elements(By.TAG_NAME, "span")
    print(pages[i].text)

    pages[i].click()
    time.sleep(1)

    models = driver.find_elements(By.CLASS_NAME, "car_list")
    for model in models:
        print("===================")
        try:
            infos = model.find_elements(By.CLASS_NAME, "cls")
            for info in infos:
                car = Info()
                brand_name = info.find_element(By.TAG_NAME, "strong")
                model_name = info.find_element(By.TAG_NAME, "em")
                car.brand_name = brand_name.text
                car.model_name = model_name.text

                print(brand_name.text)
                print(model_name.text)
                
                driver.execute_script("arguments[0].scrollIntoView();", info)
                info.click()

                time.sleep(0.5)

                """new tab opened"""
                original_window = driver.current_window_handle
                wait = WebDriverWait(driver, 2)
                wait.until(lambda d: len(d.window_handles) > 1)

                new_window = [window for window in driver.window_handles if window != original_window][0]
                driver.switch_to.window(new_window)

                try:
                    Detail = driver.find_element(By.CLASS_NAME, "DetailSummary_btn_detail__msm-h")
                    price = driver.find_element(By.CLASS_NAME, "DetailLeadCase_price__tfeps")
                    accident = driver.find_element(By.CLASS_NAME, "EncarDiagnosisPanel_link__9ZR72")
                    price = price.text.replace("만원", '')
                    price = int(price.replace(',', ''))*10000
                    car.price = price
                    car.accident = accident.text

                    print(price)
                    print(accident.text)
                    Detail.click()

                    try:
                        info_box = driver.find_element(By.ID, "bottom_sheet")
                        infos = info_box.find_element(By.TAG_NAME, "ul")
                        info = infos.find_elements(By.TAG_NAME, "li")
                        cnt = 0
                        for each in info:
                            cnt += 1
                            match cnt:
                                case 2:
                                    needs = each.find_element(By.CLASS_NAME, "DetailSpec_txt__NGapF")
                                    year = needs.text.split(" ")[0]
                                    year = int(year.replace('년', ''))
                                    car.year = year
                                    print(car.year)
                                case 3:
                                    needs = each.find_element(By.CLASS_NAME, "DetailSpec_txt__NGapF")
                                    distance = needs.text.replace("km", '')
                                    try:
                                        distance = int(distance.replace(",", ''))
                                    except:
                                        pass
                                    car.distance = distance
                                    print(car.distance)
                                case 4:
                                    needs = each.find_element(By.CLASS_NAME, "DetailSpec_txt__NGapF")
                                    cc = needs.text.replace(",", '')
                                    cc = int(cc.replace("cc", ''))
                                    car.cc = cc
                                    print(car.cc)
                                case 5:
                                    needs = each.find_element(By.CLASS_NAME, "DetailSpec_txt__NGapF")
                                    car.fueltype = needs.text
                                    print(car.fueltype)
                                case 6:
                                    needs = each.find_element(By.CLASS_NAME, "DetailSpec_txt__NGapF")
                                    car.transmission = needs.text
                                    print(car.transmission)
                                case 7:
                                    needs = each.find_element(By.CLASS_NAME, "DetailSpec_txt__NGapF")
                                    car.kind = needs.text
                                    print(car.kind)

                        try:
                            sql_text = car.sql_query()
                            cur.execute(sql_text, (car.brand_name, car.model_name, car.price, car.accident, car.year, car.distance, car.cc, car.kind, car.transmission, car.fueltype))
                            remote.commit()
                        except:
                            print("SQL Error")
                    except:
                        print("Detailed Info Error")

                except:
                    print("Price or Accident Error")
                
                del car # delete instance
                driver.close()
                driver.switch_to.window(driver.window_handles[0])
        except:
            print("Error")

In [16]:
remote.close()

---

In [18]:
import pandas as pd

query = "SELECT * FROM CarInfo;"
df = pd.read_sql(query, remote)
remote.close()

df

/tmp/ipykernel_59340/2482947094.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, remote)


,brand,name,price,accident,year,distance,cc,kind,transmission,fueltype
0,현대,더 뉴 그랜저 IG,26900000,프레임무사고 확인,20,26143,2497,대형차,오토,가솔린
1,기아,K5 3세대,27900000,프레임무사고 확인,22,20222,1598,중형차,오토,가솔린
2,기아,더 뉴 레이,12600000,프레임무사고 확인,20,15939,998,경차,오토,가솔린
3,제네시스,G80 (RG3),37500000,프레임무사고 확인,21,84011,2497,대형차,오토,가솔린
4,쉐보레(GM대우),더 넥스트 스파크,5400000,프레임무사고 확인,16,107792,999,경차,오토,가솔린
...,...,...,...,...,...,...,...,...,...,...
477,벤츠,E-클래스 W213,46990000,프레임무사고 확인,21,31579,1950,중형차,오토,디젤
478,BMW,X1 (E84),8200000,프레임무사고 확인,11,99995,1995,SUV,오토,디젤
479,미니,쿠퍼,12300000,프레임무사고 확인,15,54594,1499,소형차,오토,가솔린
480,폭스바겐,골프 7세대,8790000,프레임무사고 확인,15,142141,1395,준중형차,오토,가솔린


예약중 가격인 '9,999만원'은 df에서 삭제

In [26]:
df[df['price']==99990000]

,brand,name,price,accident,year,distance,cc,kind,transmission,fueltype
37,현대,더 뉴 코나,99990000,프레임무사고 확인,21,27598,1598,SUV,오토,가솔린
185,현대,더 뉴 아반떼 (CN7),99990000,프레임무사고 확인,23,10330,1598,준중형차,오토,가솔린
192,기아,K7 프리미어,99990000,프레임무사고 확인,21,56956,2497,대형차,오토,가솔린
228,벤츠,GLC-클래스 X253,99990000,프레임무사고 확인,22,37449,1950,SUV,오토,디젤
383,BMW,5시리즈 (G30),99990000,프레임무사고 확인,22,47292,1998,중형차,오토,가솔린
463,재규어,F-PACE,99990000,프레임무사고 확인,17,176182,1999,SUV,오토,디젤


In [28]:
df = df.drop(df[df['price']==99990000].index)

df

,brand,name,price,accident,year,distance,cc,kind,transmission,fueltype
0,현대,더 뉴 그랜저 IG,26900000,프레임무사고 확인,20,26143,2497,대형차,오토,가솔린
1,기아,K5 3세대,27900000,프레임무사고 확인,22,20222,1598,중형차,오토,가솔린
2,기아,더 뉴 레이,12600000,프레임무사고 확인,20,15939,998,경차,오토,가솔린
3,제네시스,G80 (RG3),37500000,프레임무사고 확인,21,84011,2497,대형차,오토,가솔린
4,쉐보레(GM대우),더 넥스트 스파크,5400000,프레임무사고 확인,16,107792,999,경차,오토,가솔린
...,...,...,...,...,...,...,...,...,...,...
477,벤츠,E-클래스 W213,46990000,프레임무사고 확인,21,31579,1950,중형차,오토,디젤
478,BMW,X1 (E84),8200000,프레임무사고 확인,11,99995,1995,SUV,오토,디젤
479,미니,쿠퍼,12300000,프레임무사고 확인,15,54594,1499,소형차,오토,가솔린
480,폭스바겐,골프 7세대,8790000,프레임무사고 확인,15,142141,1395,준중형차,오토,가솔린


정상 삭제 확인

In [29]:
df[df['price']==99990000]

,brand,name,price,accident,year,distance,cc,kind,transmission,fueltype


------

사고 여부가 의심스럽다. 모든 매물이 무사고일 것 같다.

In [36]:
df[df['accident']!="프레임무사고 확인"]

,brand,name,price,accident,year,distance,cc,kind,transmission,fueltype


유사고 차량이 없다. 사고 여부는 예측 성능에 도움을 주지 않을 것으로 예상된다.